In [2]:
import json
import requests
from PIL import Image
from tensorflow.keras.utils import img_to_array
import numpy as np
import tensorflow as tf

def prepare_image(src_img, target):
    if src_img.mode != "RGB":
        src_img = src_img.convert("RGB")
    src_img = src_img.resize(target)

    src_img = img_to_array(src_img)
    src_img = np.expand_dims(src_img, axis=0)

    return src_img

img_path = "IMG_D_A4_013092_bbox_000.jpg"
src_image = prepare_image(Image.open(img_path), target=(224, 224))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko'
    ,'content-type' : 'text/plain;charset=UTF-8'
}

url = 'http://localhost:8501/v1/models/test_model:predict' #
data = {"signature_name": "serving_default", "instances": src_image.tolist()}
res = requests.post(url, headers = headers, data = data) #
print(res.text)

ConnectionError: HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: /v1/models/test_model:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f6ca2859be0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [20]:
import json
import requests
from PIL import Image
from tensorflow.keras.utils import img_to_array
import numpy as np
import tensorflow as tf

def prepare_image(src_img, target):
	if src_img.mode != "RGB":
		src_img = src_img.convert("RGB")
	src_img = src_img.resize(target)

	src_img = img_to_array(src_img)
	src_img = np.expand_dims(src_img, axis=0)

	return src_img

img_path = "https://raw.githubusercontent.com/replicate/cog-examples/main/resnet/cat.png"
src_image = prepare_image(Image.open(img_path), target=(224, 224))
model.predict(src_image.tolist())

FileNotFoundError: [Errno 2] No such file or directory: 'https://raw.githubusercontent.com/replicate/cog-examples/main/resnet/cat.png'

In [31]:
#from PIL import Image
from tensorflow.keras.utils import img_to_array
import numpy as np

from PIL import Image
import urllib.request
import time
from io import BytesIO


def prepare_image(src_img, target):
    if src_img.mode != "RGB":
        src_img = src_img.convert("RGB")
    src_img = src_img.resize(target)

    src_img = img_to_array(src_img)
    src_img = np.expand_dims(src_img, axis=0)

    return src_img

url = "https://raw.githubusercontent.com/replicate/cog-examples/main/resnet/cat.png"

res = requests.get(url)
urlopen_img = Image.open(BytesIO(res.content))

src_image = prepare_image(urlopen_img, target=(224, 224))
src_image.tolist()

In [33]:
import io
import base64
import requests
from PIL import Image

# FastAPI URL
#url = "http://localhost:5000/predictions"
# 요청
url = "https://raw.githubusercontent.com/replicate/cog-examples/main/resnet/cat.png"
result = requests.get(url)
# 딕셔너리 변환
res = json.loads(result.content)
# 이미지 Bytes 리스트
bytes_list = list(map(lambda x: base64.b64decode(x), res))
# 이미지 Pillow Image 리스트
image_list = list(map(lambda x: Image.open(io.BytesIO(x)), bytes_list))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

In [ ]:
from fastapi import APIRouter
from fastapi.responses import JSONResponse
from PIL import Image
import requests
import base64
import io

router = APIRouter()

def from_image_to_bytes(img):
    """
    pillow image 객체를 bytes로 변환
    """
    # Pillow 이미지 객체를 Bytes로 변환
    imgByteArr = io.BytesIO()
    img.save(imgByteArr, format=img.format)
    imgByteArr = imgByteArr.getvalue()
    # Base64로 Bytes를 인코딩
    encoded = base64.b64encode(imgByteArr)
    # Base64로 ascii로 디코딩
    decoded = encoded.decode('ascii')
    return decoded

@router.get("/image/")
async def func():
    # 이미지 주소
    img_url = "https://dimg.donga.com/ugc/CDB/WEEKLY/Article/5b/b3/22/85/5bb32285000ed2738de6.jpg"
    # 이미지 불러오기
    img = Image.open(requests.get(img_url, stream=True).raw)
    # 이미지 데이터 타입 변환
    img_converted = from_image_to_bytes(img)
    # 이미지 리스트 생성
    img_list = [img_converted, img_converted]
    return JSONResponse(img_list)